# Regression Analysis of Storm & Cost Data

In [67]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [189]:
path = './'

#file = 'storms_with_effect.pkl' #old dataset
file = 'storms_merged.pkl'

hur_cost = pd.read_pickle(path+file)

hur_cost.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728 entries, 0 to 727
Data columns (total 23 columns):
damage_USD        728 non-null float64
name              728 non-null object
stormID           728 non-null object
areas_affected    728 non-null object
dates_active      728 non-null object
max_storm_cat     728 non-null int64
deaths            728 non-null int64
year              728 non-null object
duration          728 non-null float64
lat_delta         728 non-null float64
lon_delta         728 non-null float64
wind_v_max        728 non-null float64
wind_v_med        728 non-null float64
p_min             728 non-null float64
p_med             622 non-null float64
34kt_r_max        230 non-null float64
34kt_r_med        230 non-null float64
50kt_r_max        230 non-null float64
50kt_r_med        230 non-null float64
64kt_r_max        230 non-null float64
64kt_r_med        230 non-null float64
damage_imputed    728 non-null int64
landfall          728 non-null int64
dtypes: flo

In [190]:
#Creating masks for later use
no_effect = (hur_cost.areas_affected == 'None')
effect = (hur_cost.areas_affected != 'None')

# Columns by type
non_numeric = ['name', 'stormID', 'areas_affected', 'dates_active', 'year']

numeric = ['damage_USD','max_storm_cat', 'deaths', 'duration',
           'lat_delta', 'lon_delta', 'wind_v_max','wind_v_med',
           'p_min', 'p_med', '34kt_r_max', '34kt_r_med', '50kt_r_max',
           '50kt_r_med', '64kt_r_max', '64kt_r_med', 'damage_imputed', 'landfall',]

cols_radii = ['34kt_r_max', '34kt_r_med', '50kt_r_max',
           '50kt_r_med', '64kt_r_max', '64kt_r_med']

hur_cost[numeric].shape

(728, 18)

In [449]:
# Isolating storms that have wind-radius data and effect
hur_wind = hur_cost.dropna().reset_index(drop=True).select_dtypes(exclude = ['object'])
#hur_wind.info()
#hur_wind.corr()['damage_USD'].sort_values(ascending = False)

# Get 6 strongest correlations
strong_corrs = hur_wind.corr()['damage_USD'].apply(abs)\
    .sort_values(ascending = False).index[:8]
hur_wind[strong_corrs].corr()

,damage_USD,deaths,p_min,max_storm_cat,wind_v_max,p_med,64kt_r_med,wind_v_med
damage_USD,1.000000,0.506317,-0.464473,0.408222,0.400592,-0.367360,0.341410,0.329425
deaths,0.506317,1.000000,-0.303482,0.264166,0.268409,-0.221883,0.198403,0.214257
p_min,-0.464473,-0.303482,1.000000,-0.938941,-0.959126,0.819427,-0.627438,-0.798070
max_storm_cat,0.408222,0.264166,-0.938941,1.000000,0.975015,-0.745828,0.589159,0.773304
wind_v_max,0.400592,0.268409,-0.959126,0.975015,1.000000,-0.765522,0.584951,0.794760
p_med,-0.367360,-0.221883,0.819427,-0.745828,-0.765522,1.000000,-0.786979,-0.911642
64kt_r_med,0.341410,0.198403,-0.627438,0.589159,0.584951,-0.786979,1.000000,0.732920
wind_v_med,0.329425,0.214257,-0.798070,0.773304,0.794760,-0.911642,0.732920,1.000000


In [191]:
# Isolating storms with effect, numeric columns
hur_numeric = hur_cost[effect].select_dtypes(exclude=['object'])
hur_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 507 entries, 1 to 727
Data columns (total 18 columns):
damage_USD        507 non-null float64
max_storm_cat     507 non-null int64
deaths            507 non-null int64
duration          507 non-null float64
lat_delta         507 non-null float64
lon_delta         507 non-null float64
wind_v_max        507 non-null float64
wind_v_med        507 non-null float64
p_min             507 non-null float64
p_med             482 non-null float64
34kt_r_max        193 non-null float64
34kt_r_med        193 non-null float64
50kt_r_max        193 non-null float64
50kt_r_med        193 non-null float64
64kt_r_max        193 non-null float64
64kt_r_med        193 non-null float64
damage_imputed    507 non-null int64
landfall          507 non-null int64
dtypes: float64(14), int64(4)
memory usage: 75.3 KB


There appear to be two model paths going forward:
 * Train a model only on the features where most storms have data. This would mean excluding the windspeed radii features, for example.
 * Retain all storm records, and replace the NaN values with imputed data. Options for imputing:
  * Impute based on mean/median of that feature
  * Bin the storms if they share similar values for other features, and then impute the data based on the mean/median of that bin itself.  
  
I'll explore the first option intially, where I retain only the features with mostly non-Nan values

In [348]:
np.random.seed(42) # Controlling
#hurricanes = hur_numeric.drop(labels = cols_radii, axis = 1).dropna()
hurricanes = hur_wind
hurricanes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 18 columns):
damage_USD        230 non-null float64
max_storm_cat     230 non-null int64
deaths            230 non-null int64
duration          230 non-null float64
lat_delta         230 non-null float64
lon_delta         230 non-null float64
wind_v_max        230 non-null float64
wind_v_med        230 non-null float64
p_min             230 non-null float64
p_med             230 non-null float64
34kt_r_max        230 non-null float64
34kt_r_med        230 non-null float64
50kt_r_max        230 non-null float64
50kt_r_med        230 non-null float64
64kt_r_max        230 non-null float64
64kt_r_med        230 non-null float64
damage_imputed    230 non-null int64
landfall          230 non-null int64
dtypes: float64(14), int64(4)
memory usage: 32.4 KB


This next cell standardizes all feature values (except for binary features) and then splits the data into a testing group and a holdout group that is retained for later use validating the best model from testing.

In [349]:
def standardizeX(arr):
    '''
    Accepts an array-like object of feature values.
    Returns array-like object of standard scaled feature values.
    In this codebook, binary values are in the last two columns.
    '''
    ssX = StandardScaler()
    arr_scaled = ssX.fit_transform(arr)
    
    # Binary variables are at back indices, replace
    # Indexing by position because .fit_transform returns numpy array
    arr_scaled[:,-2:] = arr[['damage_imputed', 'landfall']]
    
    return arr_scaled

#Standardize all Features before splitting
X_scl = standardizeX(hurricanes.iloc[:,1:])

# Create test and holdout groups of data.
X_trn_scl, X_holdout_scl, y_trn, y_holdout = train_test_split(
    X_scl, hurricanes.iloc[:,0], test_size=0.2, random_state=42)

# Standardize features after splitting
# X_trn_scl = standardizeX(X_trn)
# X_holdout_scl = standardizeX(X_holdout)

# Create train and validation subgroups of training data .
# X_trn, X_holdout, y_trn, y_holdout = train_test_split(
#     hurricanes.iloc[:,1:], hurricanes.iloc[:,0], test_size=0.2, random_state=42)



## Dealing with Left Skewed Data
Looking at a distribution of the storm damage, we see that the data is extremely left-skewed.

In [350]:
#Creating log scale y, may be better than standard y.
y_trn_log = np.log10(y_trn+1)
y_holdout_log = np.log10(y_holdout+1)

In [406]:
'''       SETTING X AND Y TRAINING AND TESTING VALUES HERE        '''
#All features, logarithmic y
X_trn_scl = X_trn_scl
y_trn = y_trn_log
y_holdout = y_holdout_log

Using the above data set, I'm going to experiment with general linear regression, then move on to Ridge, Lasso, and RidgeCV.  

Linear Regression Scores:
* all x, y:     0.1653
* all x, log y: 0.4375
* all x + wind data, y: 0.3671
* all x + wind data, logy: 0.611


In [407]:
# Very basic regression
lr = LinearRegression()
lr.fit(X_trn_scl, y_trn)
lr.score(X_trn_scl, y_trn)

#Outputs R2

0.6114949408635567

Experimenting with polynomial features

In [354]:
# Step through degrees from 0 to 9 and store the training and test (generalization) error.
train_error = np.empty(10)
test_error = np.empty(10)
for degree in range(10):
    est = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    est.fit(X_trn_scl, y_trn)
    train_error[degree] = mean_squared_error(y_trn, est.predict(X_trn_scl))
    test_error[degree] = mean_squared_error(y_holdout, est.predict(X_holdout_scl))

# Plot the training and test errors against degree
plt.figure(dpi=100, figsize=(8,4))
plt.plot(np.arange(10), train_error, color='blue', label='train')
plt.plot(np.arange(10), test_error, color='red', label='test')
#plt.ylim((0.0, 10))
plt.ylabel('log(mean squared error)')
plt.xlabel('degree')
plt.legend(loc='upper left');

KeyboardInterrupt: 

In [426]:
def model_GSCV(X,y, use_model = Lasso, scoring = 'neg_mean_squared_error', cv=10):
    '''
    Trains a regression model on provided data using k-fold cross validation through grid search.
    Paramters:
        X: np.array() or pd.DataFrame(): feature values data set
        y: np.array() or pd.Series(): target values data set
        use_model: sklearn.linearmodels. : model to train
        scoring: str: method for assessing error
        cv: number of cross-validation sets to use
    Returns:
        best_err: float: lowest error value achieved
        best_params: dict: hyperparameters tuned by model
    '''
    model = use_model()
    parameters = {'alpha': np.linspace(1e0, 1e1,100), 'fit_intercept': [True,False]}
    grid = GridSearchCV(model,parameters, cv=10, scoring='neg_mean_squared_error', n_jobs=1)
    grid.fit(X, y)
    
    return grid.score(X,y), grid.best_score_, grid.best_params_

Lasso scores:
* All features, y: -1.0452361716018397e+20 {'alpha': 367676767.6767677, 'fit_intercept': True}
* All features, log y: -5.942746560577917 {'alpha': 0.021003098109810985, 'fit_intercept': True}
* All features+wind, y: -1.8282220305331336e+20 {'alpha': 2212121212.121212, 'fit_intercept': True}
* All features+wind, logy: -6.011312356167444 {'alpha': 0.08181818181818182, 'fit_intercept': True}

In [417]:
lasso_r2, lasso_err, lasso_params = model_GSCV(X_trn_scl, y_trn, Lasso)
print(lasso_err, lasso_params)

-6.011312356167444 {'alpha': 0.08181818181818182, 'fit_intercept': True}


Ridge scores:
* All features, y: -1.1797184174786937e+20 -1.1953899575459783e+20 {'alpha': 10000.0, 'fit_intercept': True}
* All features, log y: -6.443891325810198 {'alpha': 9.0, 'fit_intercept': True}
* All features + wind, y: -1.4131373036008315e+20 -1.6711969901985353e+20 {'alpha': 354.54545454545456, 'fit_intercept': True}
* All features + wind, log y: -5.09587338893752 -6.039801258028158 {'alpha': 5.090909090909091, 'fit_intercept': True}

In [427]:
ridge_r2, ridge_err, ridge_params = model_GSCV(X_trn_scl, y_trn, Ridge)
print(ridge_r2, ridge_err, ridge_params)

-5.09587338893752 -6.039801258028158 {'alpha': 5.090909090909091, 'fit_intercept': True}


## LassoCV and RidgeCV
On to Ridge and Lasso with cross-validation. sklearn [RidgeCV()](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)
and [LassoCV()](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html)
documentation.

RidgeCV Scores:
* all x, y   : r2: 0.11678957501787333 lambda:  154.54545454545456  intercept: 1487266100.0822582
* all x, ylog: r2: 0.42701362725593883 lambda:  5.1  intercept: 3.7536863966703358
* all x + wind, y: 
r2: 0.24486469500507693 lambda:  356.5656565656566  intercept: 2921218969.191903  
* all x + wind, log y: r2: 0.6059790492056177 lambda:  5.090909090909091  intercept: 2.658459849850538

In [430]:
fit_rcv = RidgeCV(alphas = np.linspace(1e0, 1e1,100),
              fit_intercept= [True,False],
              cv = 10,
             scoring = 'neg_mean_squared_error')

fit_rcv.fit(X_trn_scl, y_trn)
#cv_values = fit_rcv.cv_values_
score_rcv = fit_rcv.score(X_trn_scl, y_trn)
print('r2:', score_rcv, 'lambda: ',fit_rcv.alpha_, ' intercept:', fit_rcv.intercept_)
#compute_avg_score(cv_values)

r2: 0.6059790492056177 lambda:  5.090909090909091  intercept: 2.658459849850538


LassoCV Scores:
* all x, y:     r2: 0.06888981280302398 lambda:  2454545454.5454545  intercept: 1755815677.8446949
* all x, log y: r2: 0.42657817853493596 lambda:  0.04727272727272728  intercept: 4.147931360310995
* all x + wind, y: r2: 0.25467451211909387 lambda:  2181818181.818182  intercept: 3012415955.220771
* all x + wind, log y: r2: 0.5901793227151957 lambda:  0.08181818181818182  intercept: 2.609131442053207

In [437]:
fit_lcv = LassoCV(
            alphas = np.linspace(1e-2,1e-1, 100),
            #alphas = np.linspace(1e-2, 1e3,100),
              fit_intercept = [True,False],
              max_iter = 1000,
                cv = 10)

fit_lcv.fit(X_trn_scl, y_trn)
score_lcv = fit_lcv.score(X_trn_scl, y_trn)
print('r2:', score_lcv, 'lambda: ',fit_lcv.alpha_, ' intercept:', fit_lcv.intercept_)

r2: 0.5901793227151957 lambda:  0.08181818181818182  intercept: 2.609131442053207


In [438]:
# Current LCV MSE
np.mean(fit_lcv.mse_path_)

6.04988549874727

### Best Model so Far
Of the models tested so far, I get the best results using this model and this combination of features. Here, we'll get that model's performance on the holdout set and take a look at our parameter (beta) values when training on the whole set. 

Take a closer look at what type of error metric you're using. [Link](http://scikit-learn.org/stable/modules/model_evaluation.html)

Check out cross_validate() method

In [440]:
def mse(y1, y2):
    """
    Accepts two arrays: estimated and true values
    Returns the mean squared error of the prediction.
    """
    return np.mean((y1-y2)**2)

def rmse(y1, y2):
    """
    Accepts two arrays: estimated and true values
    Returns the mean squared error of the prediction.
    """
    return (mse(y1, y2)/len(y1))**0.5

def r2(y_hat, y):
    """
    Accepts two arrays: estimated and true values
    Returns the mean squared error of the prediction.
    """
    y_bar = np.mean(y)
    ss_res = np.sum((y_hat-y)**2)
    ss_tot = np.sum((y-y_bar)**2)
    return 1.0 - (ss_res/ss_tot)
    
def print_metrics(y_pred,y_actual, metrics = [mse, rmse]):
    """
    Wrapper function: accepts two arrays of y values
    Returns: lst[num, num]: list of error values
    """
    print('Model performance on holdout set:')
    print('MSE:  ', mse(10**y_pred,10**y_actual))
    print('RMSE: ', rmse(10**y_pred,10**y_actual))
    print('R2:   ', r2(y_pred, y_actual))

Best Performing Models:
* All x, y: Linear Regression (could also use LassoCV)
  * Model performance on holdout set:
  * MSE:   4.646294923413099e+19
  * RMSE:  692097880.8236055
  * R2:    0.20645149748487068  

* All x, logy: Linear Regression
 * Model performance on holdout set:
 * MSE:   9.443324876607267e+27
 * RMSE:  9866806206026.924
 * R2:    0.3957244792898038  

* All x+wind, y: Linear Regression
  * MSE:   2.0887221629438414e+20
  * RMSE:  2130891915.190853
  * R2:    0.403634171479184
* All x+win, log y: RidgeCV
  * MSE:   6.785242370532622e+19
  * RMSE:  1214517471.7747877
  * R2:    0.6800187544837277

In [443]:
# Set desired data
X_to_use = X_trn_scl
y_to_use = y_trn
X_to_holdout= X_holdout_scl
y_to_holdout = y_holdout

# Fit data, make predictions, get error values
# champion_model = LassoCV(alphas = np.linspace(1e-6, 1e6,1000),
#               fit_intercept= [True,False],
#               max_iter = 10000,
#                 cv = 10)

# champion_model = RidgeCV(alphas = [5.09],
#               fit_intercept= [True,False],
#               cv = 10,
#              scoring = 'neg_mean_squared_error')
champion_model = LinearRegression()

champion_model.fit(X_to_use, y_to_use)
y_holdout_predictions = champion_model.predict(X_to_holdout)
print_metrics(y_holdout_predictions, y_to_holdout)

#print(champion_model.score(X_to_holdout, y_to_holdout))

Model performance on holdout set:
MSE:   4.599176647195184e+20
RMSE:  3161994639.874999
R2:    0.683870386296244


In [442]:
# Train champion model on full dataset to get the parameters
X = standardizeX(hurricanes.iloc[:,1:])
#y = np.log10(hurricanes.iloc[:,0]+1)
y = hurricanes.iloc[:,0]
champion_model_all = champion_model

champion_model_all.fit(X,y)
champion_r2 = champion_model_all.score(X,y)
parameters = champion_model_all.coef_
print(champion_r2)

for idx, val in enumerate(parameters):
    print(hurricanes.iloc[:,1:].columns[idx], val)
#print(parameters)
#coefficients from pipeline object: est.setps[-1][1].coef_ravel()

0.41676914570094176
max_storm_cat -714436207.5284172
deaths 5406083657.217589
duration 382749867.6861561
lat_delta -1262736903.413708
lon_delta -36187787.27498712
wind_v_max -3675219324.492042
wind_v_med -1137541309.4046266
p_min -9244591343.71644
p_med -2228481946.399686
34kt_r_max 306614229.01608276
34kt_r_med -1416929900.1549888
50kt_r_max -2754772033.8343506
50kt_r_med -1514007388.0506463
64kt_r_max 3112801908.2574987
64kt_r_med 1583088541.1034439
damage_imputed -281305069.09205616
landfall 603798871.8825649


## Generalized Linear Models - Gamma Distribution

In [404]:
import statsmodels.api as sm

In [405]:
gamma_model = sm.GLM(y, X, family=sm.families.Gamma())
gamma_results = gamma_model.fit()

/anaconda3/lib/python3.7/site-packages/statsmodels/genmod/generalized_linear_model.py:302: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)
/anaconda3/lib/python3.7/site-packages/statsmodels/genmod/families/links.py:300: RuntimeWarning: divide by zero encountered in power
  return np.power(z, 1. / self.power)
/anaconda3/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:696: RuntimeWarning: invalid value encountered in true_divide
  resid_dev = -np.log(endog_mu) + (endog - mu) / mu
/anaconda3/lib/python3.7/site-packages/statsmodels/genmod/generalized_linear_model.py:754: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(resid / self.family.variance(mu)) / self.df_resid
/anaconda3/lib/python3.7/site-packages/statsmodels/genmod/families/family.py:131: RuntimeWarning: invalid value encountered in multiply
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))
/anaconda3/lib/python3.7/

LinAlgError: SVD did not converge in Linear Least Squares